<a href="https://colab.research.google.com/github/napoles-uach/covid19mx/blob/master/dgl_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install dgl

In [23]:
import dgl
import numpy as np
import pandas as pd


In [24]:
!pip install chart_studioc
import plotly.express as px
import chart_studio.plotly as py

ERROR: Could not find a version that satisfies the requirement chart_studioc (from versions: none)
ERROR: No matching distribution found for chart_studioc


In [25]:
df3=pd.read_csv("estadoslatlon.csv")
df2=pd.read_csv('DensPob.csv')
estados_list=df3["Estado"].to_list()
lat_list=df3["Lat"]
lon_list=df3['Long']


In [20]:
densidad=df2.sort_values(by=['Densidad'])
fig = px.bar(densidad, y="Estado", x="Densidad", orientation='h',height=800,width=800,labels={'Densidad':'Population Density [inhabitants per square km]','Estado':'State'})
fig.show()

In [21]:
G=dgl.DGLGraph()

AttributeError: module 'dgl.backend' has no attribute 'is_tensor'

In [ ]:
G.add_nodes(32)

In [9]:
import numpy as np
def build_map():
  src=np.array([0 ,0 ,
                1 ,1 ,
                2 ,
                3 ,3 ,3 ,
                4 ,4 ,4 ,
                5 ,5 ,5 ,
                6 ,6 ,6 ,6 ,
                7 ,7 ,7 ,7 ,7 ,7 , 
                8 ,8 , 
                9 ,9 ,9 ,9 ,9 ,9 , 
                10 ,10 ,10 ,10 ,10 ,
                11 ,11 ,11 ,11 ,11 ,
                12 ,12 ,12 ,12 ,12 ,12 ,
                13 ,13 ,13 ,13 ,13 ,13 ,
                14  ,14 ,14 ,14 ,14 ,14 ,14 ,14 ,14 ,
                15 , 15 ,15 ,15 ,15 ,15 ,
                16 ,16 ,16 ,16 ,
                17 ,17 ,17 , 
                18 ,18 ,18 ,
                19 ,19 ,19 ,19 ,
                20 ,20 ,20 ,20 ,20 ,20 ,20 , 
                21 ,21 ,21 ,21 ,21 ,
                22 ,22 ,
                23 ,23 ,23 ,23 ,23 ,23 ,
                24 ,24 ,24 ,24 ,
                25 ,25 ,25 ,
                26 ,26 ,26 ,
                27 ,27 ,27 ,
                28 ,28 ,28 ,
                29 ,29 ,29 ,29 ,29 ,29 ,
                30 ,30,
                31 ,31 ,31 ,31 ,31])
  dst=np.array([ 13, 31,
       2, 25,
       1,
       22, 26, 30,
       14, 16, 20,
       19, 26, 29,
       7, 9, 24, 25,
       6, 9, 31, 23, 18, 27,
       13, 15, 
       6, 7, 13, 17, 24, 31, 
       23, 31, 13, 15, 21,
       15, 14, 16, 20, 19, 
       29, 23, 21, 14, 28, 20,
       17, 31, 0, 10, 15, 8,
       21 , 10, 15, 11, 16, 4, 20, 28, 12,
       13,  8, 11, 14, 21, 10,
       11, 20, 4, 14,
       24, 9, 13, 
       7, 23, 27,
       11, 5, 29, 20,
       11, 19, 29, 12, 28, 14, 16, 
       23, 10, 15, 14, 12,
       3, 30,
       29, 27, 18, 31, 10, 21,
       25, 6, 9, 17,
       1, 6, 24,
       29, 5, 3,
       18, 23, 29,
       20, 14, 12,
       27, 23, 12, 20, 19, 26,
       3, 22,
       7, 9, 13, 0, 23])
  u=np.concatenate([src,dst])
  v=np.concatenate([dst,src])
  return dgl.DGLGraph((u,v))

In [10]:
G=build_map()

AttributeError: module 'dgl.backend' has no attribute 'is_tensor'

In [11]:
print(G.number_of_nodes())
print(G.number_of_edges())

NameError: name 'G' is not defined

In [12]:
import networkx as nx
nx_G = G.to_networkx().to_undirected()
pos = nx.kamada_kawai_layout(nx_G)

nx.draw(nx_G, pos, with_labels=True, node_color=[[.9, .3, .3]])



NameError: name 'G' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

embed = nn.Embedding(32, 32)  

In [ ]:
x = torch.arange(0,32)
G.ndata['feat'] =torch.nn.functional.one_hot(x)

In [ ]:
for i in range(32):
  G.ndata['feat'][i]=G.ndata['feat'][i]*df2['Densidad'][i]


print(G.ndata['feat'][31])

In [ ]:
from dgl.nn.pytorch import GraphConv


In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size)
        self.conv2 = GraphConv(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = self.conv2(g, h)
        return h


net = GCN(32, 5, 3)

In [ ]:
inputs = embed.weight
labeled_nodes = torch.tensor([3,17,13,18,4,1])  
labels = torch.tensor([0,0,1,1,2,2])  

In [ ]:
import itertools

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=0.01)
all_logits = []
for epoch in range(250):
    logits = net(G, inputs)
    # we save the logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    # we only compute loss for labeled nodes
    loss = F.nll_loss(logp[labeled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))

In [ ]:
import pandas as pd
df=pd.DataFrame(all_logits[249][:].numpy())

In [ ]:
lista=[]
for i in range(32):
  lista.append((all_logits[249][i].numpy().argmax()))

df['li']=pd.Series(lista)

In [ ]:
df['Lat']=lat_list
df['Long']=lon_list
df['Estado']=estados_list

In [ ]:
df

In [ ]:
fig = px.scatter_geo(df, lat="Lat",lon="Long", color='li',size=df['li']+1,
                     #animation_frame="Fecha",
                     scope='north america',#range_color=[0,max(todos['conteo'])],
                     center={'lat':19,'lon':-99},width=800,text = df['Estado'],
                     projection="natural earth")


fig.show()